In [1]:
import pandas as pd
import numpy as np

from pandas.tseries.offsets import DateOffset
from binance.client import Client
from datetime import datetime as dt


In [2]:
MAX_MONTHS = 24

In [ ]:
import time
def get_data():
    # take the time
    start_time = time.time()
    
    api_key = '2jxiCQ8OIWmU4PZH4xfwKEY9KYerDkSWzwNCqoaMzj41eJgWBsSqA3VYqkt2wmdX'
    api_secret = 'YY1Qj1t0JZrE4tQdaBBxT8iwl2tbFalWp1FHjyZ9selBb6OnQ0Oj8aVdiXO7YLMz'

    client = Client(api_key, api_secret)

    # Hämta handelspar
    symbols = client.get_all_tickers()
    symbols = [
        symbol for symbol in symbols if symbol['symbol'].endswith('USDT')]

    # Sätt upp en tom lista för att lagra close-priser
    close_prices = {}

    # Ange den tidsram du vill ha för historiska data
    interval = Client.KLINE_INTERVAL_1DAY
    start = f"{MAX_MONTHS} month ago UTC"

    # Hämta close-priser för alla kryptovalutor
    dates = None

    # progress_bar = st.progress(0)  # Create a progress bar

    for idx, symbol in enumerate(symbols):
        try:
            klines = client.get_historical_klines(
                symbol['symbol'], interval, start)
            if dates is None:
                # Extrahera och konvertera tidsstämplar till datum
                dates = [dt.fromtimestamp(
                    int(kline[0]) / 1000).strftime('%Y-%m-%d') for kline in klines]
            close_prices[symbol['symbol']] = [
                float(kline[4]) for kline in klines]
        except Exception as e:
            print(f"Kunde inte hämta data för {symbol['symbol']}: {e}")

        # Update the progress bar
        if idx+1 == len(symbols):
            progress_text = f"Done! last symbol . . . . . . . . {symbol['symbol']} fetched."
        else:
            progress_text = f"This will take several minutes. Symbol number {idx+1} of {len(symbols)}: . . . . . . . . {symbol['symbol']}"
        # progress_bar.progress((idx + 1) / len(symbols), progress_text)

    # Konvertera close_prices-dikten till en pandas DataFrame
    df = pd.DataFrame.from_dict(close_prices, orient='index').transpose()
    
    df = df.dropna(axis=1,how='all')  # remove columns with all nan
    
    # Lägg till datum som index för DataFrame
    df.index = pd.to_datetime(dates)  # type: ignore
    # print(df.head())
    
    # take the time    
    print("--- %s seconds ---" % round((time.time() - start_time),1))
    return df

df = get_data()
df.to_csv('all_tickers.csv', index=False)


In [95]:

df = pd.read_csv('all_tickers.csv')

print(df.shape)
# drop all columns where all values are nan
df = df.dropna(axis=1,how='all')
print(df.shape)

df[df.isna().any(axis=1)]

s=0.66
# remove all columns where there are more than s% nan values
    
df = df.dropna(axis=1,thresh=int(s*len(df.columns)))

df.shape

(730, 442)
(730, 416)


(730, 348)

In [100]:
int(730*(s))


481